In [1]:
import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session



sess = Session()

role = get_execution_role()
model_dir = "/home/ec2-user/SageMaker/smoke-detection-smokeynet/src" # Replace with S3

In [2]:
pt_mnist_model_data = sess.upload_data(
    path="/home/ec2-user/SageMaker/smoke-detection-smokeynet/src/last.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch"
)

In [3]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir=model_dir,
    role=role,
    model_data=pt_mnist_model_data,
    framework_version="1.5.0",
    py_version="py3",
)

#### Deploy Model

In [4]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.g4dn.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

---------!

In [5]:
import random
import numpy as np
import boto3
import io
import pickle
dummy_data = {"inputs": np.random.rand(2, 45, 2, 3, 224, 224).tolist()}
s3 = boto3.client("s3")
bucket = "smokynet-inference-images-processed"
my_array_data = io.BytesIO()
pickle.dump(dummy_data, my_array_data)
my_array_data.seek(0)
s3.upload_fileobj(my_array_data, bucket, 'mydummydata.pkl')

NameError: name 's3_client' is not defined

In [ ]:
arr = np.random.rand(1, 1, 2, 3, 224, 224)
arr.size * arr.itemsize

In [6]:


res = predictor.predict('bucket_name' : 'smokynet-inference-images-processed','file_name' :'mydummydata.pkl')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "shape '[1, 1, 155, 279]' is invalid for input of size 961
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_inference/transformer.py", line 126, in transform
    result = self._transform_fn(self._model, input_data, content_type, accept)
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_inference/transformer.py", line 216, in _default_transform_fn
    prediction = self._predict_fn(data, model)
  File "/opt/ml/model/code/inference.py", line 91, in predict_fn
    prediction = model(input_object)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/ml/model/code/inference.py", line 27, in forward
    outputs, x = sub_model(x, bbox_labels=bbox_labels, tile_outputs=outputs)
  File "/opt/conda/lib/python3

In [ ]:
!pip install transformers

#### Delete endpoint

In [ ]:
import os

if not local_mode:
    predictor.delete_endpoint()
else:
    os.system("docker container ls | grep 8080 | awk '{print $1}' | xargs docker container rm -f")